In [1]:
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
%ls

LJ_data/                        high-dimension-search-results/
LJ_data.tar                     images/
README.md                       jobs/
__pycache__/                    low-dimension-search-results/
algorithm_compare_results/      models/
data/                           paper-images/
data-queue/                     requirements.txt
data-queue-2023-09-24/          results/
data-queue-2023-09-27/          search_results/
deeplifting/                    tasks.py
deeplifting.png                 test-low-dimension-results.png
high-dimension-paper-results/


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.models import DeepliftingSkipMLP
from deeplifting.optimization import deeplifting_predictions
from deeplifting.utils import initialize_vector

In [17]:
problem = PROBLEMS_BY_NAME['bukin_n6']
output_size = problem['dimensions']

# Set up the deeplifting model
input_size = 1
hidden_sizes = (128,) * 5
bounds = problem['bounds']
activation = 'sine'
output_activation = 'leaky_relu'
agg_function = 'sum'
trial = 0

model = DeepliftingSkipMLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
    bounds=bounds,
    skip_every_n=1,
    activation=activation,
    output_activation=output_activation,
    agg_function=agg_function,
    seed=trial,
)

# Need to setup an objective
results = np.zeros((1, 1, 3))
trial = 0

objective = problem['objective']
fn = lambda x: objective(x, results=results, trial=trial, version='pytorch')

In [18]:
inputs = torch.randn(1, 5 * output_size).to(dtype=torch.double)
model = model.to(dtype=torch.double)
outputs = model(inputs=inputs)
outputs.flatten()

tensor([-6.8324,  2.1061], dtype=torch.float64,
       grad_fn=<ReshapeAliasBackward0>)

In [19]:
x0 = initialize_vector(size=output_size, bounds=bounds)
x0 = torch.from_numpy(x0)
x0 = x0.to(dtype=torch.double)
print(x0)

tensor([-9.5119,  1.2911], dtype=torch.float64)


In [20]:
# We only want to manipulate the last linear layer weights
for name, parameters in model.named_parameters():
    if 'output' not in name:
        parameters.requires_grad = False

In [21]:
# Now let's train the weights to get this outpu
model.train()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(inputs=inputs)
    outputs = outputs.flatten()
    loss = criterion(x0, outputs)
    loss.backward()
    optimizer.step()


print(outputs)

tensor([-9.5119,  1.2911], dtype=torch.float64,
       grad_fn=<ReshapeAliasBackward0>)


In [23]:
# Now make sure all of the model weights are trainable
for name, parameters in model.named_parameters():
    if 'output' not in name:
        parameters.requires_grad = True

In [24]:
model(inputs=inputs)

tensor([[-9.5119,  1.2911]], dtype=torch.float64, grad_fn=<StackBackward0>)

In [ ]:
# We only want to manipulate the last linear layer weights
for name, parameters in model.named_parameters():
    if 'output' not in name:
        parameters.requires_grad = False

# Now let's train the weights to get this outpu
model.train()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(inputs=inputs)
    outputs = outputs.flatten()
    loss = criterion(x0, outputs)
    loss.backward()
    optimizer.step()

# Now make sure all of the model weights are trainable
for name, parameters in model.named_parameters():
    if 'output' not in name:
        parameters.requires_grad = True

In [27]:
completed_experiment = {
    'low-dimension': {'ackley': 'Complete', 'bukin_n6': 'Complete'},
    'high-dimension': {'ackley': {'ackley-3d': 'complete'}},
}

running_experiment = {'2d': {'ackley': 'Complete'}}

completed_experiment == running_experiment

False